In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u

In [2]:
def get_name(file):
    if 'pz_act' in file:
        return file[5:13]
    return file[5:9]

In [3]:
def calc_error(det_cat, true_cat, shift=15/60, match_dist=5/60, n_try=20, seed=0):
    import numpy as np
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    
    error = []
    np.random.seed(seed)
    for i in range(n_try):
        det_sc = SkyCoord(ra=np.array(det_cat['RA']) * u.degree, 
                          dec=np.array(det_cat['DEC']) * u.degree, frame='icrs')
        angles = np.random.randint(0, 360, len(det_cat))
        det_sc = det_sc.directional_offset_by(angles*u.degree, shift)

        true_sc = SkyCoord(ra=np.array(true_cat['RA']) * u.degree, 
                           dec=np.array(true_cat['DEC']) * u.degree, frame='icrs')
        _, d2d, _ = det_sc.match_to_catalog_sky(true_sc)
        c_error = np.count_nonzero(d2d.degree < match_dist)
        error.append(c_error)
    error = np.array(error)
    return error.mean(), error.std() / np.sqrt(n_try - 1)

In [26]:
def gen_tables(det_cats_files, true_cats_files, z_cut = -1, m_cut=-1, b_cut=None, match_dist=5/60):
    
    true_cats = {os.path.splitext(os.path.basename(file))[0] : pd.read_csv(file) for file in true_cats_files}
    det_cats = {get_name(os.path.splitext(os.path.basename(file))[0]) : 
                pd.read_csv(file) for file in det_cats_files}
    
    comp_df = []
    recall_df = []
    
    for det_name in det_cats:
        det = det_cats[det_name]
        det = det[det['status'] != 'fn']
        det.index = np.arange(len(det))
        line = {}
        line_r = {}

        det_sc = SkyCoord(ra=np.array(det['RA'])*u.degree, 
                      dec=np.array(det['DEC'])*u.degree, frame='icrs')
        det['b'] = det_sc.galactic.b.degree
        
        if not(b_cut is None):
            det = det[det['b'] > b_cut]
            det.index = np.arange(len(det))
            det_sc = SkyCoord(ra=np.array(det['RA'])*u.degree, 
                          dec=np.array(det['DEC'])*u.degree, frame='icrs')
            

        for tr_name in true_cats:
            
            if tr_name == 'RM' and (z_cut > -1 or m_cut > -1):
                continue
            tr = true_cats[tr_name]
            if z_cut > -1:    
                tr = tr[tr['z'] > z_cut]
            if m_cut > -1:
                tr = tr[tr['M500'] > m_cut]
                
            tr_sc = SkyCoord(ra=np.array(tr['RA'])*u.degree, 
                          dec=np.array(tr['DEC'])*u.degree, frame='icrs')
            tr['b'] = tr_sc.galactic.b.degree
            true_cats[tr_name] = tr
            if not(b_cut is None):
                tr = tr[tr['b'] > b_cut]
                tr.index = np.arange(len(tr))
                tr_sc = SkyCoord(ra=np.array(tr['RA'])*u.degree, 
                              dec=np.array(tr['DEC'])*u.degree, frame='icrs')
            
            tr.index = np.arange(len(tr))
            
            idx, d2d, _ = det_sc.match_to_catalog_sky(tr_sc)
            matched = d2d.degree <= match_dist
            
            line[tr_name] = np.count_nonzero(matched)
            line[tr_name+'_err'], line[tr_name+'_std'] = calc_error(det, tr)

            line_r[tr_name] = line[tr_name] / len(tr)
            
        line['all'] = len(det)
        line['fp'] = np.count_nonzero(det['status'] == 'fp')
        line_r['fp'] = line['fp']
        line_r['all'] = line['all']
        comp_df.append(pd.DataFrame(line, index=[det_name]))
        recall_df.append(pd.DataFrame(line_r, index=[det_name]))
        line = {}
    
    for tr_name in true_cats:
        if tr_name == 'RM' and (z_cut > -1 or m_cut > -1):
            continue
        tr = true_cats[tr_name]
        if z_cut > -1:    
            tr = tr[tr['z'] > z_cut]
        if m_cut > -1:
            tr = tr[tr['M500'] > m_cut] 
        tr.index = np.arange(len(tr))
        
        line[tr_name] = len(tr)
        line[tr_name+'_err'] = 0
    line['fp'] = 0
    line['all'] = 0
    comp_df.append(pd.DataFrame(line, index=['all']))
    
    comp_df = pd.concat(comp_df)
    recall_df = pd.concat(recall_df)
    
    return comp_df, recall_df

In [15]:
tr_dir = '/home/rt2122/Data/original_catalogs/csv/'
true_cats_files = [os.path.join(tr_dir, name) for name in next(os.walk(tr_dir))[-1]]
det_dir = '/home/rt2122/Data/detected_cats/'
det_cats_files = [os.path.join(det_dir, name) for name in sorted(next(os.walk(det_dir))[-1]) if 'full' in name]
#det_cats_files = ['/home/rt2122/Data/detected_cats/full_pz14_thr0.1_step8.csv', 
#                 '/home/rt2122/Data/detected_cats/full_pz_act10_thr0.1_step8.csv']
comp, recall = gen_tables(det_cats_files, true_cats_files)
comp

<ipython-input-10-f18a654f857f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det['b'] = det_sc.galactic.b.degree


,RSZ2,RSZ2_err,RSZ2_std,MCXC,MCXC_err,MCXC_std,RM,RM_err,RM_std,ACT,ACT_err,ACT_std,all,fp
pz14,1491,15.75,1.073300,725,17.95,0.838153,1242,298.75,3.305398,849,43.25,1.389576,17968,15828
pz20,1528,20.30,1.076300,740,20.55,0.910393,1306,311.90,5.114427,875,50.30,1.592912,25312,23104
pz25,1525,19.20,0.741975,747,20.15,0.998090,1374,341.00,4.090425,888,53.65,1.351948,22837,20611
pz40,1506,18.15,1.103285,739,20.20,0.878396,1279,328.65,3.937856,871,49.90,1.470589,19495,17306
pz_act10,1361,13.20,0.936398,659,13.65,0.785644,1029,206.60,3.642873,926,30.65,1.076727,18363,16316
pz_act14,1363,15.10,0.953939,670,13.80,0.902044,1211,260.45,3.811668,1282,39.75,1.305605,18878,16484
pz_act20,1217,10.30,0.696230,599,10.50,0.626183,1182,191.30,3.585864,1866,33.55,1.298734,12239,9398
pz_act25,1260,14.00,0.954215,610,13.40,1.003677,1271,208.55,2.953566,2152,37.90,1.454394,18447,15275
all,1653,0.00,NaN,1743,0.00,NaN,26111,0.00,NaN,4195,0.00,NaN,0,0


In [17]:
recall

,RSZ2,MCXC,RM,ACT,fp,all
pz14,0.901996,0.415950,0.047566,0.202384,15828,17968
pz20,0.924380,0.424555,0.050017,0.208582,23104,25312
pz25,0.922565,0.428571,0.052622,0.211681,20611,22837
pz40,0.911071,0.423982,0.048983,0.207628,17306,19495
pz_act10,0.823351,0.378084,0.039409,0.220739,16316,18363
pz_act14,0.824561,0.384395,0.046379,0.305602,16484,18878
pz_act20,0.736237,0.343660,0.045268,0.444815,9398,12239
pz_act25,0.762250,0.349971,0.048677,0.512992,15275,18447


In [18]:
comp, recall = gen_tables(det_cats_files, true_cats_files, m_cut=4)
comp

<ipython-input-10-f18a654f857f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det['b'] = det_sc.galactic.b.degree
<ipython-input-10-f18a654f857f>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr['b'] = tr_sc.galactic.b.degree


,RSZ2,RSZ2_err,RSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,702,7.65,0.726473,224,2.50,0.380443,260,3.20,0.401314,17968,15828
pz20,711,9.35,0.662034,222,3.95,0.450000,260,5.00,0.453060,25312,23104
pz25,710,8.30,0.487205,220,3.30,0.317059,268,4.35,0.519489,22837,20611
pz40,705,9.15,0.775530,221,3.25,0.452333,264,5.20,0.484496,19495,17306
pz_act10,660,5.75,0.648379,219,1.95,0.358909,249,2.00,0.316228,18363,16316
pz_act14,667,6.55,0.642671,222,2.30,0.333246,267,3.45,0.505106,18878,16484
pz_act20,634,4.50,0.380443,215,1.75,0.298020,275,2.40,0.335606,12239,9398
pz_act25,644,5.65,0.534470,217,2.30,0.370632,283,3.30,0.333246,18447,15275
all,713,0.00,NaN,264,0.00,NaN,342,0.00,NaN,0,0


In [19]:
recall

,RSZ2,MCXC,ACT,fp,all
pz14,0.984572,0.848485,0.760234,15828,17968
pz20,0.997195,0.840909,0.760234,23104,25312
pz25,0.995792,0.833333,0.783626,20611,22837
pz40,0.988780,0.837121,0.771930,17306,19495
pz_act10,0.925666,0.829545,0.728070,16316,18363
pz_act14,0.935484,0.840909,0.780702,16484,18878
pz_act20,0.889201,0.814394,0.804094,9398,12239
pz_act25,0.903226,0.821970,0.827485,15275,18447


In [20]:
comp, recall = gen_tables(det_cats_files, true_cats_files, m_cut=4, z_cut=0.5)
comp

<ipython-input-10-f18a654f857f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det['b'] = det_sc.galactic.b.degree


,RSZ2,RSZ2_err,RSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,94,1.20,0.312881,11,0.20,0.116980,72,1.00,0.261574,17968,15828
pz20,98,0.85,0.243602,11,0.15,0.081918,76,1.75,0.298020,25312,23104
pz25,98,1.10,0.289282,11,0.30,0.146898,81,1.75,0.260314,22837,20611
pz40,95,1.15,0.232549,11,0.10,0.068825,81,2.10,0.306937,19495,17306
pz_act10,85,0.50,0.153897,11,0.10,0.100000,68,0.45,0.135239,18363,16316
pz_act14,86,0.80,0.247088,11,0.05,0.050000,82,1.45,0.336194,18878,16484
pz_act20,78,0.60,0.152177,9,0.05,0.050000,91,0.85,0.254176,12239,9398
pz_act25,81,0.85,0.195677,11,0.20,0.091766,97,1.40,0.244949,18447,15275
all,97,0.00,NaN,13,0.00,NaN,132,0.00,NaN,0,0


In [21]:
recall

,RSZ2,MCXC,ACT,fp,all
pz14,0.969072,0.846154,0.545455,15828,17968
pz20,1.010309,0.846154,0.575758,23104,25312
pz25,1.010309,0.846154,0.613636,20611,22837
pz40,0.979381,0.846154,0.613636,17306,19495
pz_act10,0.876289,0.846154,0.515152,16316,18363
pz_act14,0.886598,0.846154,0.621212,16484,18878
pz_act20,0.804124,0.692308,0.689394,9398,12239
pz_act25,0.835052,0.846154,0.734848,15275,18447


In [22]:
comp, recall = gen_tables(det_cats_files, true_cats_files, m_cut=5)
comp

<ipython-input-10-f18a654f857f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det['b'] = det_sc.galactic.b.degree
<ipython-input-10-f18a654f857f>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr['b'] = tr_sc.galactic.b.degree


,RSZ2,RSZ2_err,RSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,486,5.25,0.561366,118,1.55,0.266310,142,1.75,0.227977,17968,15828
pz20,489,7.00,0.640723,117,2.40,0.365629,144,2.50,0.373462,25312,23104
pz25,489,5.90,0.390007,116,1.95,0.294467,145,1.95,0.328273,22837,20611
pz40,485,5.85,0.558546,118,1.45,0.266310,142,2.35,0.292674,19495,17306
pz_act10,463,4.20,0.521132,118,0.85,0.254176,133,0.85,0.220943,18363,16316
pz_act14,466,5.35,0.498814,118,1.30,0.218849,136,1.30,0.241704,18878,16484
pz_act20,448,2.70,0.230560,115,0.90,0.216430,135,1.05,0.198348,12239,9398
pz_act25,455,4.15,0.436945,116,1.45,0.328273,138,1.45,0.223312,18447,15275
all,490,0.00,NaN,137,0.00,NaN,153,0.00,NaN,0,0


In [23]:
recall

,RSZ2,MCXC,ACT,fp,all
pz14,0.991837,0.861314,0.928105,15828,17968
pz20,0.997959,0.854015,0.941176,23104,25312
pz25,0.997959,0.846715,0.947712,20611,22837
pz40,0.989796,0.861314,0.928105,17306,19495
pz_act10,0.944898,0.861314,0.869281,16316,18363
pz_act14,0.951020,0.861314,0.888889,16484,18878
pz_act20,0.914286,0.839416,0.882353,9398,12239
pz_act25,0.928571,0.846715,0.901961,15275,18447


In [27]:
comp, recall = gen_tables(det_cats_files, true_cats_files, b_cut=20)
comp

<ipython-input-26-1eba5f218421>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det['b'] = det_sc.galactic.b.degree


,RSZ2,RSZ2_err,RSZ2_std,MCXC,MCXC_err,MCXC_std,RM,RM_err,RM_std,ACT,ACT_err,ACT_std,all,fp
pz14,591,6.10,0.593385,332,9.50,0.748683,916,202.35,3.354710,241,19.30,0.987021,7112,6266
pz20,608,7.10,0.496832,341,9.60,0.563822,940,206.95,3.350943,245,18.75,0.790985,7329,6458
pz25,601,6.85,0.488149,344,9.00,0.676679,1011,227.00,3.728976,242,18.90,0.981138,7946,7082
pz40,597,6.70,0.607410,343,9.70,0.781699,945,215.60,2.574572,251,19.80,1.340856,7678,6810
pz_act10,538,3.75,0.415964,306,7.05,0.617614,758,143.65,2.900340,295,13.50,0.738063,4772,3952
pz_act14,548,5.10,0.475173,311,8.80,0.713037,891,186.25,2.853322,468,18.80,0.869362,5963,4958
pz_act20,511,4.20,0.456532,281,6.20,0.432861,848,134.90,2.871732,681,15.35,1.119857,4385,3206
pz_act25,513,4.50,0.366348,276,5.80,0.526158,882,147.45,2.329982,772,14.85,1.044472,4988,3718
all,1653,0.00,NaN,1743,0.00,NaN,26111,0.00,NaN,4195,0.00,NaN,0,0


In [28]:
recall

,RSZ2,MCXC,RM,ACT,fp,all
pz14,0.917702,0.362050,0.051414,0.164057,6266,7112
pz20,0.944099,0.371865,0.052762,0.166780,6458,7329
pz25,0.933230,0.375136,0.056747,0.164738,7082,7946
pz40,0.927019,0.374046,0.053042,0.170865,6810,7678
pz_act10,0.835404,0.333697,0.042546,0.200817,3952,4772
pz_act14,0.850932,0.339149,0.050011,0.318584,4958,5963
pz_act20,0.793478,0.306434,0.047598,0.463581,3206,4385
pz_act25,0.796584,0.300981,0.049506,0.525528,3718,4988
